# Programação funcional

## Funções como objetos de primeira classe

[Wikipedia: First-Class_function](https://en.wikipedia.org/wiki/First-class_function)

Python trata funções como objetos de primeira classe. Ou seja: funções podem ser passadas como argumentos, retornadas como valor, e atribuidas a variáveis. Compare com outras linguagens:

- Java: funções não existem, apenas classes e objetos. Em Java, sempre que queremos trabalhar com o conceito de funções como objetos de primeira classe devemos simular este conceito através de classes. (Em Java 8 temos lambdas, mas estes são construídos através desta idéia de simular funções com classes.)

- C: Podemos simular funções de primeira classe através do uso de ponteiros para funções.

- Assembler: Que função? Só existe ```CALL``` e ```RET```!

Mas o que isso significa? Vamos entender na prática. Considere uma função que soma 1 ao valor passado e retorna este resultado.

In [1]:
def soma_um(x):
    return x + 1


print(soma_um(12))

13


Podemos atribuir esta função a uma variável!

In [2]:
minha_funcao = soma_um

print(minha_funcao(12))

13


Podemos também passar esta função como argumento de outras funções:

In [5]:
def aplique_funcao(func, valor, repeticoes=1):
    print(f"O valor passado é {valor}")
    for i in range(repeticoes):
        valor = func(valor)
        print(f"aplicando {i + 1} vez(es), o resultado é {valor}")
    print(f"O valor retornado será {valor}")
    return valor


aplique_funcao(soma_um, 12, 5)

O valor passado é 12
aplicando 1 vez(es), o resultado é 13
aplicando 2 vez(es), o resultado é 14
aplicando 3 vez(es), o resultado é 15
aplicando 4 vez(es), o resultado é 16
aplicando 5 vez(es), o resultado é 17
O valor retornado será 17


17

Em Python podemos criar uma função em qualquer parte de nosso código.

In [6]:
print("Antes de criar a função")


def triplica_valor(x):
    return 3 * x


print("Agora temos uma nova função.")
print(triplica_valor(11))

Antes de criar a função
Agora temos uma nova função.
33


Podemos inclusive criar uma função dentro de uma função:

In [7]:
import random


def my_sorting_function(vec):
    def my_inner_sorting_function(vec, inicio, fim):
        if inicio == fim:
            return

        def particiona(vec, inicio, fim):
            pivot = vec[inicio]
            meio = inicio + 1
            final_atual = inicio + 1

            def troca(vec, i, j):
                aux = vec[i]
                vec[i] = vec[j]
                vec[j] = aux

            while final_atual < fim:
                # Invariante:
                # vec[(inicio + 1) : meio] < pivot
                # vec[meio : final_atual] >= pivot

                # Se o novo elemento for menor que o pivot, ele pertence à partição inferior.
                if vec[final_atual] < pivot:
                    troca(vec, final_atual, meio)
                    meio += 1

                final_atual += 1

            # Posiciona o pivot na sua posição correta.
            troca(vec, inicio, meio - 1)

            return meio - 1

        p = particiona(vec, inicio, fim)
        my_inner_sorting_function(vec, inicio, p)
        my_inner_sorting_function(vec, p + 1, fim)

    my_inner_sorting_function(vec, 0, len(vec))

In [8]:
my_vec = [3, 1, 4, 2]
my_sorting_function(my_vec)
my_vec

[1, 2, 3, 4]

In [9]:
def test_sorting_function(
    sorting_function,
    num_tests=1000,
    vec_size=101,
    rand_range=100,
):
    def is_sorted(x):
        for i in range(len(x) - 1):
            if x[i] > x[i + 1]:
                return False
        return True

    for i in range(num_tests):
        vec = [random.randrange(rand_range) for x in range(vec_size)]
        sorting_function(vec)
        if not is_sorted(vec):
            return False

    return True


print(test_sorting_function(my_sorting_function))

True


## Lambda

Podemos criar funções anônimas em Python, chamadas *lambda*. Lambdas são normalmente usados quando precisamos de uma função simples, que se resume a um *statement*. Veja este exemplo:

In [10]:
def mapeia(func, vec):
    resultado = []
    for value in vec:
        resultado.append(func(value))
    return resultado


def quadrado(x):
    return x * x


data = [2, 3, 5, 7]
res = mapeia(quadrado, data)

print(data)
print(res)

[2, 3, 5, 7]
[4, 9, 25, 49]


Note que a função ```quadrado()``` é muito simples. Nestes casos podemos usar um lambda:

In [11]:
res2 = mapeia(lambda x: x * x, data)
print(res2)

[4, 9, 25, 49]


Veja como o lambda funciona: usamos a *keyword* ```lambda``` seguida dos argumentos da função e um dois-pontos. Em seguida vem uma expressão, que será avaliada e produzirá o valor retornado. Podemos usar mais de um argumento, como no exemplo abaixo:

In [12]:
def reduz(func, vec, valor_inicial):
    valor = valor_inicial
    for item in vec:
        valor = func(item, valor)
    return valor


vec = list(range(10))
soma = reduz(lambda x, y: x + y, vec, 0)

print(vec)
print(soma)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
45


Funções lambda não são essenciais à programação em Python, fica a seu cargo usar lambdas ou funções com nome. Mesmo o criador da linguagem Python (Guido van Rossum, o "*Benevolent Dictator For Life*" do Python) se arrepende de ter criado lambdas, e queria te-los removido do Python 3, mas a comunidade em geral acha lambdas úteis. Fica a seu cargo, não existe regra geral para adotar ou rejeitar o uso de lambdas.

### Exercício

Faça uma função ```filtra(func, vec)``` que recebe uma função ```func(valor)``` e uma lista ```vec```, e retorna uma lista com os valores de ```vec``` para os quais ```func(valor)``` é ```True```.

In [13]:
def filtra(func, vec):
    novo_vec = []
    for item in vec:
        if func(item):
            novo_vec.append(item)
    return novo_vec


x = list(range(10))
print(x)


def par(a):
    return a % 2 == 0


y = filtra(par, x)
print(y)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 2, 4, 6, 8]


## Closures

https://www.programiz.com/python-programming/closure

Talvez você tenha percebido por acidente que em um escopo qualquer de Python temos acesso às variáveis do escopo e de escopos externos. Por exemplo:

In [12]:
x = 5


def func(value):
    return x + value


print(func(3))

x = 7
print(func(3))

8
10


Isso normalmente é fonte de complicações, e deve ser evitado a todo custo! Existe uma exceção: *closures*.

Em Python, *closure* é o nome dado à uma construção muito particular:

- Temos uma função definida dentro de uma função.
- Esta função interna usa variáveis do escopo da função externa.
- A função externa retorna a função interna.

Confuso? Vejamos um exemplo:

In [14]:
# Eis um exemplo de closure
def aumentador(incremento):
    def _aumentador(x):
        return x + incremento

    return _aumentador


soma_um = aumentador(1)
soma_cinco = aumentador(5)
print(soma_um(3))

4


In [15]:
# A partir de agora podemos até mesmo destruir aumentador()
del aumentador

# Veja que aumentador não existe mais:
try:
    aux = aumentador(3)
except NameError as e:
    print(e)

name 'aumentador' is not defined


In [16]:
# Porém soma_um() continua firme!
print(soma_um(5))
print(soma_cinco(5))

6
10


Em um closure o sistema faz um "backup" das variáveis do escopo externo, **mas apenas nas condições de closure**. Confira os atributos de ```soma_um``` (afinal funções são objetos):

In [17]:
dir(soma_um)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Observe que temos um atributo "```__closure__```". Vejamos o que está contido neste atributo:

In [18]:
dir(soma_cinco.__closure__)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

In [19]:
len(soma_cinco.__closure__)

1

In [20]:
dir(soma_cinco.__closure__[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'cell_contents']

In [21]:
soma_cinco.__closure__[0].cell_contents

5

Observe que o valor foi guardado no closure.

Ok, mas e daí? Para que servem os *closures*?

### Decorators

In [22]:
# Eis o decorator
def temp_dec(nome):
    def imprime_args(func):
        def func_wrapper(*args, **kwargs):
            resultado = func(*args, **kwargs)
            print("Argumentos posicionais: {}".format(args))
            print("Argumentos nomeados: {}".format(kwargs))
            print("Comentário: {}".format(nome))
            print("Resultado: {}".format(resultado))
            return resultado

        return func_wrapper

    return imprime_args


# Aplicação do decorator
@temp_dec("Insper")
def uma_funcao_qualquer(x, y, z):
    return x + y + z


p = uma_funcao_qualquer(1, 2, z=3)
print(p)

Argumentos posicionais: (1, 2)
Argumentos nomeados: {'z': 3}
Comentário: Insper
Resultado: 6
6


### Exercício

Caching: faça um decorator para guardar um dicionário de valores já calculados de uma função. Se em uma nova chamada tivermos como argumento um valor já visto, retorna direto do dicionário, senão realmente chama a função sendo decorada.

In [23]:
def caching():
    def closure(f):
        cache = {}

        def func_wrapper(*args, **kwargs):
            val = args[0]
            if val in cache:
                print(f"Retornando valor cacheado de {val}")
                resultado = cache[val]
            else:
                print(f"Tenho que calcular para {val}")
                resultado = f(*args, **kwargs)
                print(f"Retornou ({val}) -> ({resultado})")
                cache[val] = resultado
            return resultado

        return func_wrapper

    return closure

In [24]:
# Aplicação do decorator
@caching()
def fatorial(x):
    print(f"Calculando fat({x})")
    return 1 if x <= 0 else x * fatorial(x - 1)


print("-" * 50)
print(f"Fatorial(4) = {fatorial(4)}")
print("-" * 50)
print(f"Fatorial(2) = {fatorial(2)}")
print("-" * 50)
print(f"Fatorial(6) = {fatorial(6)}")
print("-" * 50)

--------------------------------------------------
Tenho que calcular para 4
Calculando fat(4)
Tenho que calcular para 3
Calculando fat(3)
Tenho que calcular para 2
Calculando fat(2)
Tenho que calcular para 1
Calculando fat(1)
Tenho que calcular para 0
Calculando fat(0)
Retornou (0) -> (1)
Retornou (1) -> (1)
Retornou (2) -> (2)
Retornou (3) -> (6)
Retornou (4) -> (24)
Fatorial(4) = 24
--------------------------------------------------
Retornando valor cacheado de 2
Fatorial(2) = 2
--------------------------------------------------
Tenho que calcular para 6
Calculando fat(6)
Tenho que calcular para 5
Calculando fat(5)
Retornando valor cacheado de 4
Retornou (5) -> (120)
Retornou (6) -> (720)
Fatorial(6) = 720
--------------------------------------------------


Este padrão é tão comum que o Python já oferece esse decorator no pacote ```functools```. Veja este exemplo obtido da documentação da biblioteca:

In [25]:
import functools


@functools.lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


[fib(n) for n in range(16)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [26]:
fib.cache_info()

CacheInfo(hits=28, misses=16, maxsize=None, currsize=16)

### Aplicação parcial de argumentos

Eis um *closure* que recebe uma função que opera em dois argumentos (uma $f(x,y)$) e um valor $x_0$ e "congela" o primeiro valor, resultando em uma $g(y) = f(x_0,y)$".

In [27]:
def aplica_primeiro_argumento(func, x):
    def wrapper(y):
        return func(x, y)

    return wrapper

Testando:

In [28]:
def soma_dois_args(x, y):
    return x + y


print(soma_dois_args(2, 5))

soma_cinco = aplica_primeiro_argumento(soma_dois_args, 5)
print(soma_cinco(2))

7
7


A aplicação parcial de argumentos é uma ferramenta tão comum em programação funcional que o Python tem uma função mais genérica para isso no pacote ```functools```:

In [29]:
soma_3 = functools.partial(soma_dois_args, 3)
print(soma_3(10))

13


## List comprehensions

Antes de prosseguirmos na exploração de programação funcional em Python, vamos aprender sobre *list comprehensions*. Considere a função abaixo que retorna uma nova lista onde cada item é o quadrado do item equivalente na lista passada como argumento.

In [30]:
def lista_quadrado(vec):
    res = []
    for item in vec:
        res.append(item ** 2)
    return res


vec = [2, 3, 5]
vec_quad = lista_quadrado(vec)

print(vec_quad)

[4, 9, 25]


Vamos fazer o mesmo com *list comprehension*:

In [31]:
vec_quad_2 = [x ** 2 for x in vec]
print(vec_quad_2)

[4, 9, 25]


In [32]:
vec_quad_3 = []
for x in vec:
    vec_quad_3.append(x * x)
print(vec_quad_3)

[4, 9, 25]


Podemos também filtrar uma lista:

In [33]:
vec = list(range(20))
vec_even = [x for x in vec if x % 2 == 0]
print(vec_even)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


É possível, mas não muito recomendável, fazer multiplos `for` em um *list comprehension*:

In [34]:
res = [(x, y) for x in [1, 2, 3] for y in ["a", "b"]]
res

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]

Isto é equivalente a:

In [35]:
res = []
for x in [1, 2, 3]:
    for y in ["a", "b"]:
        res.append((x, y))
res

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]

Podemos inclusive estabelecer uma relação de dependência do segundo `for` em relação ao primeiro:

In [36]:
res = [(i, j) for i in range(3) for j in range(i, 3)]
res

[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]

Isto é equivalente a:

In [37]:
res = []
for i in range(3):
    for j in range(i, 3):
        res.append((i, j))
res

[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]

Podemos tambem incluir condicionais:

In [38]:
res = [(i, j) for i in range(5) if i % 2 == 0 for j in range(i, 5) if j % 2 == 0]
res

[(0, 0), (0, 2), (0, 4), (2, 2), (2, 4), (4, 4)]

Isto é equivalente à:

In [39]:
res = []
for i in range(5):
    if i % 2 == 0:
        for j in range(i, 5):
            if j % 2 == 0:
                res.append((i, j))
res

[(0, 0), (0, 2), (0, 4), (2, 2), (2, 4), (4, 4)]

## ```zip()```

Outra característica útil de Python é a função ```zip()```. Esta função recebe dois iteráveis e retorna um novo iterável formado por pares obtidos através da fusão elemento-a-elemento dos iteráveis iniciais. O iterável de pares que é retornado itera até que um dos iteráveis originais esteja esgotado. Por exemplo:

In [40]:
x1 = [2, 3, 5]
x2 = ["abobora", "batata", "chuchu", "tomate"]

y = zip(x1, x2)
print(y)

In [41]:
print(list(y))

[(2, 'abobora'), (3, 'batata'), (5, 'chuchu')]


In [42]:
print([x for x in zip(x1, x2)])

[(2, 'abobora'), (3, 'batata'), (5, 'chuchu')]


In [43]:
for primo, comida in zip(x1, x2):
    print(f"{comida}, {primo}")

abobora, 2
batata, 3
chuchu, 5


### Exercício

Use ```sum()```, ```zip()``` e *list comprehension* para implementar o produto escalar de dois vetores em $\mathbb{R}^n$:

$$\left<x, y\right> = \sum_{i=1}^{n} x_i y_i$$

In [44]:
x = list(range(10))
y = list(range(10))

print(x)
print(y)

dot_prod = sum([a * b for a, b in zip(x, y)])
print(dot_prod)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
285


Voltamos agora à programação original.

## Funções puras

Uma função pura é uma função sem *efeitos colaterais* (*side-effects*): não altera a entrada, nem resulta em respostas diferentes para chamadas iguais.

Considere a função a seguir:

In [45]:
def soma_um_em_tudo(vec):
    res = vec
    for i in range(len(res)):
        res[i] += 1
    return res


x = [1, 2, 3]
y = soma_um_em_tudo(x)

print(x)
print(y)

[2, 3, 4]
[2, 3, 4]


Observe que a chamada da função resultou na modificação do vetor de entrada! Esta é uma função com efeitos colaterais.

Considere agora esta outra versão:

In [46]:
def soma_um_em_tudo(vec):
    res = vec.copy()
    for i in range(len(res)):
        res[i] += 1
    return res


x = [1, 2, 3]
y = soma_um_em_tudo(x)

print(x)
print(y)

[1, 2, 3]
[2, 3, 4]


Agora tudo bem, temos uma função pura.

Veja este outro exemplo:

In [47]:
contador = 0


def foo(x):
    global contador
    contador += 1
    return x + contador


print(foo(1))
print(foo(1))

2
3


Este também é um exemplo de função com efeitos colaterais.

Funções puras são importantes porque:

- São mais fáceis de debugar;
- Podem ser cacheadas externamente;
- Podem ser paralelizáveis;
- Permitem demonstrar, em certos casos, que o sistema funciona matematicamente.

## Higher-order functions

Funções que recebem outras funções como argumento são chamadas de funções de ordem superior (*higher-order functions*). Algumas das mais importantes são ```map()```, ```filter()``` e ```reduce()```.

### ```map()```

A função ```map(func, iteravel)``` recebe uma função (preferencialmente pura) ```func``` e uma estrutura de dados iterável (como uma lista), e retorna um iterador onde cada elemento corresponde a um elemento do iterável inicial após aplicação da função. Por exemplo:

In [48]:
vec = [2, 3, 5]
aux = map(lambda x: x ** 2, vec)
print(aux)

In [49]:
resultado = list(aux)
print(resultado)

[4, 9, 25]


Note que poderíamos ter escrito um loop for para obter esse resultado. Com o map() não precisamos escrever loops (uma vantagem), e podemos deixar - em princípio - o Python decidir se quer rodar essa operação de mapeamento em paralelo, dividindo o trabalho entre vários *cores* (uma IMENSA vantagem!). 

### ```filter()```

A função ```filter(func, iteravel)``` recebe uma função ```func``` e um iterável e retorna um iterável cujos itens são aqueles onde ```func()``` retornou True ao ser aplicada aos itens do iteravel original. Por exemplo:

In [50]:
vec = list(range(10))
aux = filter(lambda x: x % 2 == 1, vec)
print(aux)

In [51]:
resultado = list(aux)
print(resultado)

[1, 3, 5, 7, 9]


### ```reduce()```
A função ```functools.reduce(func, iterable, initial)``` recebe uma função ```func(x,y)```, um iterável e um valor inicial, e combina os valores do iterável através da aplicação repetida de ```func()``` que serve para combinar os valores dois-a-dois, exatamente como na função ```reduz()``` acima. Se o valor inicial não for passado então o primeiro valor do iterável servirá para iniciar o processo. Por exemplo:

In [52]:
from functools import reduce

vec = list(range(10))
soma = reduce(lambda x, y: x + y, vec)
print(soma)

45


### Exercício

Escreva um programa que calcula o produto do valor absoluto dos elementos de uma lista, para os valores não-nulos apenas. Não use loops: use map(), filter() e reduce().

In [53]:
vec = list(range(-4, 5))
print(vec)

[-4, -3, -2, -1, 0, 1, 2, 3, 4]


In [54]:
# resultado_esperado: 1**2 + 2**2 + 3**2 + 4**2 = 1 + 4 + 9 + 16 = 30
reduce(
    lambda x, y: x + y,
    map(
        lambda x: x ** 2,
        filter(
            lambda x: x > 0,
            vec,
        ),
    ),
    0,
)

30

Como você pode ver, ficou meio esquisito mas funciona. Vamos agora construir uma classe auxiliar que permite encadear as operações:

In [55]:
class MeuDataFrameCaseiro:
    def __init__(self, dados):
        self.dados = dados

    def map(self, func):
        novos_dados = list(map(func, self.dados))
        return MeuDataFrameCaseiro(novos_dados)

    def filter(self, func):
        novos_dados = list(filter(func, self.dados))
        return MeuDataFrameCaseiro(novos_dados)

    def reduce(self, func, init):
        return reduce(func, self.dados, init)

Usando essa classe fica mais intuitivo escrever nossa tarefa acima:

In [57]:
df = MeuDataFrameCaseiro(vec)

df.filter(lambda x: x > 0).map(lambda x: x ** 2).reduce(lambda x, y: x + y, 0)

30

### Exercício

O código abaixo le as linhas de um arquivo CSV contendo dados sobre pacientes e se estes apareceram nas suas consultas médicas agendadas ou se faltaram. Escreva um programa que calcula a média de idade dos homens que faltaram às consultas. Não use loops, use apenas programação funcional.

In [58]:
import functools

import numpy as np

filename = "pacientes.csv"

with open(filename, "r", encoding="utf-8") as f:
    name_row = f.readline().strip().split(",")
    data = [x.strip().split(",") for x in f]

for i, name in enumerate(name_row):
    print("{}: {}".format(i, name))

0: PatientId
1: AppointmentID
2: Gender
3: ScheduledDay
4: AppointmentDay
5: Age
6: Neighbourhood
7: Scholarship
8: Hipertension
9: Diabetes
10: Alcoholism
11: Handcap
12: SMS_received
13: No-show


In [59]:
print(data[0])

['29872499824296', '5642903', 'F', '2016-04-29T18:38:08Z', '2016-04-29T00:00:00Z', '62', 'JARDIM DA PENHA', '0', '1', '0', '0', '0', '0', 'No']


In [60]:
df = MeuDataFrameCaseiro(data)
res = (
    df.map(lambda x: (float(x[5]), x[13]))
    .filter(lambda x: x[1] == "No")
    .map(lambda x: (1, x[0]))
    .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]), (0, 0.0))
)

In [61]:
media = res[1] / res[0]
media

37.790064393252315